# Parte 4: Clasificación de Tickets usando NLP

El modelado busca desarrollar clasificadores que automaticen el triage de tickets de soporte a partir del texto libre. Esta etapa es fundamental para traducir la información no estructurada en decisiones útiles para el negocio. Los objetivos principales son:

- Entrenar modelos supervisados que aprendan a predecir la categoría de un ticket en base a su descripción textual.
- Comparar dos enfoques: uno tradicional basado en TF-IDF + modelo clásico (Logistic Regression) y otro moderno basado en el fine-tuning de DistilBERT.
- Evaluar el rendimiento de ambos enfoques utilizando métricas como precisión, recall y F1-score.
- Detectar fortalezas y limitaciones de cada método para entender cuál se adapta mejor al problema.
- Identificar oportunidades de mejora y ajustes en el pipeline de procesamiento y modelado.
- Seleccionar el modelo más robusto para su futura implementación como sistema automatizado de clasificación.

### Importar librerías y Configuración Inicial

In [59]:
!pip install transformers torch

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import pandas as pd
import numpy as np
import re
import torch
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import tensorflow as tf

### Carga de datos y división en train/test

In [77]:
# Cargar el dataset limpio
data_entrenamiento = pd.read_csv('C:\CC219-TP-TF-2024-2--CC92\data\data_limpia2.csv')

# Revisar el dataset
print("Dimensiones del dataset:", data_entrenamiento.shape)
print("\nPrimeras filas del dataset:")
data_entrenamiento.head()

<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
C:\Users\alexa\AppData\Local\Temp\ipykernel_20924\3191167572.py:2: SyntaxWarning: invalid escape sequence '\C'
  data_entrenamiento = pd.read_csv('C:\CC219-TP-TF-2024-2--CC92\data\data_limpia2.csv')


Dimensiones del dataset: (8469, 5)

Primeras filas del dataset:


,Ticket Description,Ticket Priority,Ticket Channel,Urgency,Resolution_Time_Bin
0,billing code appreciate requested website addr...,Critical,Social media,1,0
1,need change existing product facing intermitte...,Critical,Chat,1,0
2,facing problem turning working fine yesterday ...,Low,Social media,0,0
3,problem youre interested love see happen check...,Low,Social media,0,0
4,note seller responsible damages delivery game ...,Low,Email,0,0


In [ ]:
# 4.1.1 Definir variables
# - X_urgency: columnas de descripción, prioridad y canal.
# - y_urgency: etiqueta binaria de urgencia.
# - X_time: mismas columnas para tiempo de respuesta.
# - y_time: etiqueta binaria de tiempo de resolución.

# 4.1.2 División de datos
# - Separar cada par (X, y) en train/test (80/20) con stratify para mantener proporciones.

# Entrenamiento de Modelos

### Pipeline clásico: TF‑IDF + Regresión Logística

##### 1. Definición del preprocesador

In [ ]:
# `ColumnTransformer` que aplique:
    # TF-IDF en `Ticket Description`.
    # One-Hot Encoding en variables categóricas adicionales.

(explicación de que se hace y si se imprime algo significa)

##### 2.  Construcción del pipeline

In [ ]:
# Encadenar preprocesador con `LogisticRegression`.

(explicación de que se hace y si se imprime algo significa)

##### 3.  Entrenamiento

In [ ]:
# Ajustar el pipeline a los datos de entrenamiento.

(explicación de que se hace y si se imprime algo significa)

##### 4.  Evaluación

In [ ]:
# Generar predicciones en test.
# Calcular métricas: accuracy, precision, recall, F1-score y ROC-AUC.

(explicación de que se hace y si se imprime algo significa)

#### Insight 1: Priorización ineficiente  
Al ajustar `class_weight='balanced'` en el clasificador, reforzamos la detección de tickets críticos.  
**Resultados:** _(completar con métricas: precision, recall, F1 para la clase “Urgente”)_

#### Insight 2: Desempeño por canal  
La inclusión de `Ticket Channel` permite al modelo adaptar sus predicciones según el canal de entrada.  
**Resultados:** _(completar con comparación de métricas por canal o ejemplos de casos “Chat” vs. “Email”)_

#### Insight 3: Temas recurrentes  
TF-IDF resalta las palabras clave que anticipan urgencia, capturando problemas frecuentes.  
**Resultados:** _(completar con lista de términos más frecuentes o imagen de nube de palabras)_


### Pipeline moderno: DistilBERT Fine‑Tuning

##### 1. Preparación de datos

In [ ]:
# Tokenizar descripciones con `DistilBertTokenizer`.
# Codificar etiquetas con `LabelEncoder`.

(explicación de que se hace y si se imprime algo significa)

##### 2. Preparación de datasets de entrenamiento

In [ ]:
# Generar `tf.data.Dataset` (TensorFlow) o `DataLoader` (PyTorch) a partir de los encodings y etiquetas
# Convertir encodings y etiquetas en `tf.data.Dataset` o `DataLoader` de PyTorch.

(explicación de que se hace y si se imprime algo significa)

##### 3. Configuración del modelo

In [ ]:
# Cargar `TFDistilBertForSequenceClassification` con número de clases.
# Compilar (TF) o definir optimizador (PyTorch).

(explicación de que se hace y si se imprime algo significa)

##### 4. Entrenamiento del modelo

In [ ]:
# Fine-tuning por varias épocas con validación.

(explicación de que se hace y si se imprime algo significa)

##### 5. Evaluación

In [ ]:
# Evaluar en el conjunto de prueba y reportar pérdida y exactitud.

(explicación de que se hace y si se imprime algo significa)

#### 4.3.4 Insight 1: Priorización de respuesta rápida  
El modelo de tiempo de resolución predice si un ticket recibirá respuesta en menos de una hora o no.  
**Resultados:** _(completar con métricas: precision, recall, F1 para la clase “<1h”)_

#### 4.3.5 Insight 2: Impacto del canal en la latencia  
Incluir `Ticket Channel` permitió al modelo capturar diferencias en la rapidez de atención según el canal (por ejemplo, “Chat” vs. “Email”).  
**Resultados:** _(completar con comparación de métricas por canal)_

#### 4.3.6 Insight 3: Patrones de demora  
TF‑IDF identifica términos asociados a demoras frecuentes (p. ej. “network”, “update”, “delay”).  
**Resultados:** _(completar con lista de palabras clave o visualización de frecuencia)_

### Comparativa de resultados

In [ ]:
# Métricas clave (precision, recall, F1, accuracy, ROC‑AUC) para cada enfoque.
# Matriz de confusión de ambos modelos.
# Graficar barras o tablas con precision, recall y F1-score.

(explicación de cada resultado, tiempo de entrenamiento y requisitos de hardware.)

### Guardado del modelo seleccionado

In [ ]:
# Según la comparativa, decidir el mejor modelo para producción.
# Guardar el pipeline clásico con `joblib.dump`.
# Guardar el modelo DistilBERT con `model.save_pretrained` y el tokenizer.